In [ ]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import make_balance, convert_to
from cryptotrader.agents import apriori
from cryptotrader.utils import array_normalize

from time import time, sleep

from bokeh.io import output_notebook
from jupyterthemes import jtplot
output_notebook()
jtplot.style()
%matplotlib inline
%load_ext line_profiler

import matplotlib.pyplot as plt

In [ ]:
# Environment setup
obs_steps = 300
period = 30
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP", "USDT_XMR", "USDT_ETC", "USDT_DASH"]
init_funds = make_balance(crypto=0.0, fiat=100.0, pairs=pairs)

tapi = Poloniex()
tapi = BacktestDataFeed(tapi, period, pairs, init_funds)
tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
                       start=datetime.timestamp(datetime.utcnow() - timedelta(days=200)))

env = BacktestEnvironment(period, obs_steps, tapi, 'backtest_env')
env.add_pairs(pairs)
env.fiat = 'USDT'
obs = env.reset()

In [ ]:
# Training run
# Train params
nb_steps = 100
batch_size = 1
nb_max_episode_steps = 7

mom = apriori.MomentumTrader(ma_span=[133,234], std_span=39, mean_type='kama')
fib = apriori.HarmonicTrader()

agent = apriori.FactorTrader([mom, fib])

search_space = {
                'std_window':[2, env.obs_steps],
                'std_weight':[0.0001, 3],
                'alpha_up':[1e-4, 1],
                'alpha_down':[1e-4, 1]
                }

opt_params, info = agent.fit(env, nb_steps, batch_size, search_space, nb_max_episode_steps=nb_max_episode_steps)
print('\n', opt_params, '\n', env.status)
agent.test(env, verbose=True)
env.plot_results();

In [ ]:
# Validation run
tapi.download_data(end=datetime.timestamp(datetime.now() - timedelta(days=50)),
                       start=datetime.timestamp(datetime.now() - timedelta(days=100)))
agent.test(env, verbose=True)
env.plot_results();

In [ ]:
# Test run
tapi.download_data(end=datetime.timestamp(datetime.now()),
                       start=datetime.timestamp(datetime.now() - timedelta(days=50)))
agent.test(env, verbose=True)
env.plot_results();